## Overview

In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
# maximum nr of frames to process
maxFrames = 60
# the size of the sliding window
windowSize = 30
# the magnifaction factor
factor = 20
# the fps used for the bandpass
fpsForBandPass = 600 # use -1 for input video fps
# low ideal filter
lowFreq = 72
# high ideal filter
highFreq = 92
# output video filename

In [4]:
import os, sys
sys.path.append('..')
from phasebasedMoMag import phaseBaseMagnify
from pathlib import Path
video_folder = Path('..') / 'media'
out_folder = Path('.')

ImportError: cannot import name phaseBaseMagnify

In [ ]:
video_paths = list(video_folder.glob('*.mp4'))
print(video_paths)

In [ ]:
for vidFname in video_paths:
    vidFnameOut = out_folder / '{}-Mag{}Ideal-lo{}-hi{}.mp4'(vidFname.stem, factor, lowFreq, highFreq)
    phaseBasedMagnify(str(vidFname), str(vidFnameOut), 
                      maxFrames, windowSize, factor, fpsForBandPass, lowFreq, highFreq)